### 고객 분석
※ Output은 지운 상태입니다.

**EDA 보기 전**  
기초부터...

0-1. `.csv` 파일 읽기

In [ ]:
import pandas as pd

# Warning 방지
data_type = {
    "고객번호": str
}

members = pd.read_csv('../data/members.csv', dtype=data_type)
members.head()

In [ ]:
members.dtypes

0-2. 연령별 히스토그램

In [ ]:
members = members.dropna()
ages = members["연령"]
ages

In [ ]:
from matplotlib import pyplot as plt

plt.hist(ages, bins=range(5, 75, 1))
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

0-2-a. 30대가 많은데 몇 퍼센트나 될까? 

In [ ]:
# 30 ~ 39세
ages.loc[(30 <= ages) & (ages <= 39)].size / ages.size * 100

0-2. 연령과 구매력 분포도

In [ ]:
import pandas as pd

# Warning 방지
data_type = {
    "주문번호": str,
    "고객번호": str
}

sales = pd.read_csv('../data/sales.csv', dtype=data_type)
sales.head()

In [ ]:
# 고객번호별 구매금액
user_overall_costs = sales.groupby("고객번호")["구매금액"].sum()
user_overall_costs

In [ ]:
# 고객 테이블과 병합
joined = pd.merge(left=user_overall_costs, right=members.dropna(), how='inner', on="고객번호")
joined

In [ ]:
# 2차원 히트맵 생성
plt.figure(figsize=(8, 6))
plt.scatter(joined["연령"], joined["구매금액"], s=100)
plt.xlabel('Age')
plt.ylabel('Buy cost')
plt.grid(True)
plt.show()

이것으로는 관계를 찾기 힘들 것 같다..

0-3. 단순 분석

In [ ]:
# 카테고리 바꿔 가면서 체크
members.groupby("성별").count()

In [ ]:
# 2중 카테고리
members.groupby(["성별", "결혼유무"]).count()

In [ ]:
# 기혼자 중 자녀여부 비율 확인
married_baby = members[(members["결혼유무"] == "기혼") & (members["자녀여부"] == "Yes")]
married = members[members["결혼유무"] == "기혼"]

married_baby.size / married.size * 100

In [ ]:
nonmarried_baby = members[(members["결혼유무"] == "미혼") & (members["자녀여부"] == "Yes")]
nonmarried_baby.size

In [ ]:
# 특정 상품을 구매한 고객
haggis = sales[sales["상품명"].str.contains("하기스")]
haggis.size / sales.size * 100

In [ ]:
# 그 중 자녀를 가진 고객이 구매한 비율
baby_bought = haggis[haggis["고객번호"].isin(married_baby["고객번호"])]
baby_bought.size / haggis.size * 100

**이후 EDA를 보고 오긴 했음...**

In [ ]:
members["연령"].describe()

1-1. 기혼 여부와 남성 비율

In [ ]:
pd.crosstab(index = members['성별'], columns = members['결혼유무'])

In [ ]:
import scipy.stats

"""
변수의 독립 여부 확인
귀무가설: 잠정적으로 옳은 가설, 기존의 가설. 보통 보수적으로 채택
  - 독립성 테스트의 경우, "변수 간 독립성이 없다"가 귀무가설
대립가설: 증명하고자 하는 가설. 귀무가설과 반대, 
  - 독립성 테스트의 경우, "변수 간 독립성이 있다"가 대립가설
  
p는 P-value인데 이것이 0.05보다 작으면 보통 귀무가설을 채택하여 독립이 아니라고 판단한다.
"""
def click_abtest(a_selected, total_a, b_selected, total_b):
    selected = [a_selected, b_selected]
    no_selected = [total_a-a_selected, total_b-b_selected]
    chi2, p, dof, expected = scipy.stats.chi2_contingency([selected, no_selected])
    return p

print(click_abtest(2099, 2099+66430, 397, 397+12168))

0.05보다 훨씬 큰 값으로 관련이 없다.

1-2. 기혼과 특정 상품 구매 여부

In [ ]:
def get_unique_user_count(df: pd.DataFrame) -> int:
    return df.size

haggis = sales[sales["상품명"].str.contains("하기스")]
married_sales = sales[sales["고객번호"].isin(married_baby["고객번호"])]
married_sales_haggis = married_sales[married_sales["상품명"].str.contains("하기스")]

arr = [married_sales_haggis, married_sales, haggis, sales]
counts = list(map(get_unique_user_count, arr))
print(counts)
print(click_abtest(*counts))

구매 내역 수로 해서 관련이 없는 걸까? 유저 수로 해 보자

In [ ]:
haggis["주문번호"].unique()

In [ ]:
def get_unique_user_count(df: pd.DataFrame) -> int:
    return df["주문번호"].unique().size

uniques = list(map(lambda x: get_unique_user_count(x), arr))
print(uniques)
print(click_abtest(*uniques))

역시 관련이 없다..

1-3. 체험팩?

In [ ]:
sales.groupby("물품대분류").count()

체험팩이 생각보다 비중이 높은데?

In [ ]:
experiments = sales[sales["물품대분류"] == "체험팩"]
experiments

In [ ]:
plt.hist(experiments["구매금액"], bins=range(0, 30000, 300))
plt.xlabel('구매금액')
plt.ylabel('Count')
plt.show()

In [ ]:
# Join
joined2 = pd.merge(left=experiments, right=members.dropna(), how='inner', on="고객번호")
joined2

In [ ]:
# 특정 연령층이 체험팩을 많이 샀을까?
joined2["연령"].describe()

In [ ]:
plt.hist(joined2["연령"], bins=range(5, 50, 1))
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

연령은 아닌 것 같다..

In [ ]:
# 카테고리 변경해 가며 찾기
joined2.groupby("유입경로").count()

## Other...

In [ ]:
non_haggis = sales[~ sales["상품명"].str.contains("하기스")]
non_haggis

**문제 인지**
- 분석 방법이 숙련되지 않아서인가?
  - 2개 항목이 아닌 3개 이상을 엮어야 하는가?
  - 그렇다면 데이터 프로세싱을 어떻게 해야 하는가?
- 일단 이런 과정이 EDA인 것 자체는 맞는 것 같은데...
- NaN, ??? 등의 비유효 데이터를 Clansing하지 않아서 그런가?
  - 기준은? DA의 마음대로?